<a href="https://colab.research.google.com/github/JachuPro23/automation_robotics_lab/blob/master/Lab_auto_3_Ko%C5%82odziejczyk_Szl%C4%99k_Rumas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

### REPORT LAB 3
### Kacper Kołodziejczyk 250185
### Jan Szlęk 250195
### Sewer Rumas 250191

# Integer Programming

Integer programming is a mathematical optimization technique used when some or all of the variables in a linear programming problem must take integer values. When all variables must be either 0 or 1 (binary), we call it Binary Integer Programming.

## Common applications of Integer Programming:

1. **Facility location problems** - deciding where to build facilities
2. **Resource allocation** - assigning limited resources to tasks
3. **Scheduling** - allocating time slots to activities
4. **Transportation and logistics** - optimizing routes and shipments
5. **Manufacturing** - production planning and inventory control
6. **Financial planning** - portfolio optimization with discrete investments

## Example 1: Facility Location Problem

Our company already has factories in two cities (City A and City B) and is considering expanding them. We also want to build a warehouse (only one).

Decision variables:
- Build factory in City A? (x1): Added value: $9 million, capital required: $6 million
- Build factory in City B? (x2): Added value: $5 million, capital required: $3 million
- Build warehouse in City A? (x3): Added value: $6 million, capital required: $5 million
- Build warehouse in City B? (x4): Added value: $4 million, capital required: $2 million

Available capital: $10 million

Additional constraint: A warehouse can only be built in a city if there is also a factory there.




![image](https://github.com/AdoHaha/automation_robotics_lab/blob/master/lab3a_expl.png?raw=1)


In [ ]:
    # Let's visualize the problem:
    #
    #                 City A                  City B
    #                 ------                 ------
    # Factory:     $9M value              $5M value
    #              $6M cost               $3M cost
    #                  |                      |
    #                  v                      v
    # Warehouse:   $6M value              $4M value
    #              $5M cost               $2M cost
    #
    # Total capital available: $10M
    # Must choose max 1 warehouse, and only where factory exists

In [ ]:
import pulp
from pulp import *

# Create the model
prob = LpProblem("Facility_Location_Problem", LpMaximize)

In [ ]:
# Decision variables (binary: 0 or 1)
x1 = LpVariable("Factory_CityA", cat=LpBinary)
x2 = LpVariable("Factory_CityB", cat=LpBinary)
x3 = LpVariable("Warehouse_CityA", cat=LpBinary)
x4 = LpVariable("Warehouse_CityB", cat=LpBinary)

In [ ]:
# Objective function: maximize total value
prob += 9*x1 + 5*x2 + 6*x3 + 4*x4, "Total_Added_Value"

In [ ]:
# Constraints
# 1. Capital constraint
prob += 6*x1 + 3*x2 + 5*x3 + 2*x4 <= 10, "Available_Capital"

# 2. Only one warehouse
prob += x3 + x4 <= 1, "Maximum_One_Warehouse"

# 3. Warehouse can only be built if factory exists in same city
prob += x1 - x3 >= 0, "CityA_Warehouse_Requires_Factory"
prob += x2 - x4 >= 0, "CityB_Warehouse_Requires_Factory"

In [ ]:
# Helper function to print the solution
def print_solution(p):
    p.solve()
    print("Status:", LpStatus[p.status])
    for v in p.variables():
        print(v.name, "=", v.varValue)
    print("Objective value =", value(p.objective))

In [ ]:
# Solve the problem
print_solution(prob)

Status: Optimal
Factory_CityA = 1.0
Factory_CityB = 1.0
Warehouse_CityA = 0.0
Warehouse_CityB = 0.0
Objective value = 14.0


## Binary Problems

Binary integer programming is used for "yes/no" decision problems such as:
- Which route to choose
- Which truck to use
- Whether to make an investment

We can also model more complex logical conditions using binary variables.

## Example 2: Modeling Logical Constraints

Sometimes we need to model that ONE of two constraints must be satisfied, but not necessarily both.

For example, maximize x + y where 0 ≤ x ≤ 10, 0 ≤ y ≤ 10, and EITHER (x + y ≤ 3) OR (3y + x ≤ 3).

We can use a binary variable z to implement this logical OR:

In [ ]:
prob2 = LpProblem("Alternative_Constraints_Problem", LpMaximize)

# Continuous variables
x = LpVariable("x", 0, 10, cat=LpContinuous)
y = LpVariable("y", 0, 10, cat=LpContinuous)

# Binary variable to implement logical OR
z = LpVariable("ignore_first_constraint", cat=LpBinary)

# Objective function
prob2 += x + y, "Simple_Sum"

# If z=1, first constraint is relaxed (ignored)
# If z=0, second constraint is relaxed (ignored)
M = 10000  # A very large number
prob2 += x + y <= 3 + M*z, "first_constraint"
prob2 += 3*x - y <= 3 + M*(1-z), "second_constraint"

# Solve and display results
print_solution(prob2)
print(prob2)

Status: Optimal
ignore_first_constraint = 1.0
x = 4.3333333
y = 10.0
Objective value = 14.3333333
Alternative_Constraints_Problem:
MAXIMIZE
1*x + 1*y + 0
SUBJECT TO
first_constraint: - 10000 ignore_first_constraint + x + y <= 3

second_constraint: 10000 ignore_first_constraint + 3 x - y <= 10003

VARIABLES
0 <= ignore_first_constraint <= 1 Integer
x <= 10 Continuous
y <= 10 Continuous



## Example 3: Scheduling Problem

Let's consider a scheduling problem where we need to assign employees to workdays.

### Problem description:
- We have three employees: Anna, Kate, and Peter
- Each employee can work up to 3 days per week
- Daily rates are: Anna ($150), Kate ($160), Peter ($140)
- Anna can't work on Monday, Peter can't work on Thursday and Friday
- On Tuesday we need 2 employees, on other days we need 1 employee
- Goal: minimize the total cost

This is a perfect application for integer programming with binary variables.

In [ ]:
prob3 = LpProblem("Scheduling_Problem", LpMinimize)

# Define data
days = ["mon", "tue", "wed", "thu", "fri"]
employees = ["Anna", "Kate", "Peter"]
costs = [150, 160, 140]  # daily rates

# Create binary decision variables for each employee on each day
schedule = LpVariable.dicts("Schedule", (employees, days), cat="Binary")

In [ ]:
# Objective function: minimize total cost
prob3 += lpSum([costs[i] * lpSum([schedule[employee][day] for day in days])
                for i, employee in enumerate(employees)])

In [ ]:
# Constraint: required number of employees each day
required_employees = [1, 2, 1, 1, 1]  # mon, tue, wed, thu, fri
for day, required in zip(days, required_employees):
    prob3 += lpSum([schedule[employee][day] for employee in employees]) == required

# Constraint: employee availability
prob3 += schedule["Anna"]["mon"] == 0  # Anna can't work Monday
prob3 += schedule["Peter"]["thu"] == 0  # Peter can't work Thursday
prob3 += schedule["Peter"]["fri"] == 0  # Peter can't work Friday

# Constraint: maximum workdays per employee
for employee in employees:
    prob3 += lpSum([schedule[employee][day] for day in days]) <= 3

In [ ]:
# Solve and print the solution
print_solution(prob3)

Status: Optimal
Schedule_Anna_fri = 1.0
Schedule_Anna_mon = 0.0
Schedule_Anna_thu = 1.0
Schedule_Anna_tue = 1.0
Schedule_Anna_wed = 0.0
Schedule_Kate_fri = 0.0
Schedule_Kate_mon = 0.0
Schedule_Kate_thu = 0.0
Schedule_Kate_tue = 0.0
Schedule_Kate_wed = 0.0
Schedule_Peter_fri = 0.0
Schedule_Peter_mon = 1.0
Schedule_Peter_thu = 0.0
Schedule_Peter_tue = 1.0
Schedule_Peter_wed = 1.0
Objective value = 870.0


# Excercise 1

1) Extend the example to four Emploeeys: there is also Mark, who can work on any day and his rate is the lowest: $100 per day but he can work up to 4 days per week.

2) Add a constraint(s) that Mark can work only if Anna is not there (use logical constraint)

3) Bonus:  Make the program interactive: add a checkbox to show which employees are available (and which are not)


In [ ]:
# Install PuLP if needed (uncomment when running in a new environment)
# !pip install pulp

from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpStatus, value

# Define problem
prob3 = LpProblem("Scheduling_Problem", LpMinimize)

# Data
days = ["mon", "tue", "wed", "thu", "fri"]
employees = ["Anna", "Kate", "Peter", "Mark"]
costs = [150, 160, 140, 100]  # Corresponds to Anna, Kate, Peter, Mark

# Create decision variables
schedule = LpVariable.dicts("Schedule", (employees, days), cat="Binary")

# Objective: Minimize total cost
prob3 += lpSum([costs[i] * lpSum([schedule[employee][day] for day in days])
                for i, employee in enumerate(employees)])

# Daily staffing requirements
required_employees = [1, 2, 1, 1, 1]  # mon, tue, wed, thu, fri
for day, required in zip(days, required_employees):
    prob3 += lpSum([schedule[employee][day] for employee in employees]) == required

# Employee availability constraints
prob3 += schedule["Anna"]["mon"] == 0
prob3 += schedule["Peter"]["thu"] == 0
prob3 += schedule["Peter"]["fri"] == 0

# Max workdays per employee
for employee in employees:
    max_days = 4 if employee == "Mark" else 3
    prob3 += lpSum([schedule[employee][day] for day in days]) <= max_days

# Logical constraint: Mark can work only if Anna is not working
for day in days:
    prob3 += schedule["Mark"][day] <= 1 - schedule["Anna"][day]

# Solve
prob3.solve()

# Print solution
print(f"Status: {LpStatus[prob3.status]}")
for employee in employees:
    for day in days:
        if schedule[employee][day].varValue == 1:
            print(f"{employee} works on {day}")
print(f"Total Cost = ${value(prob3.objective)}")


Status: Optimal
Peter works on tue
Peter works on wed
Mark works on mon
Mark works on tue
Mark works on thu
Mark works on fri
Total Cost = $680.0


###EXPLANATION

In this exercise, We modeled a weekly employee scheduling problem using binary decision variables to minimize labor costs while fulfilling daily staffing needs and honoring constraints.

#### Problem Setup

We have **4 employees**: Anna, Kate, Peter, and Mark. Each has a different daily cost:
- Anna: \$150
- Kate: \$160
- Peter: \$140
- Mark: \$100

The week is modeled as 5 days: **Monday to Friday**.

#### Objective

The goal is to **minimize total labor cost** while meeting the required number of workers for each day.

#### Decision Variables

Binary variables are used to define the schedule:

Schedule[employee][day] = 1 if employee works on that day, else 0


#### Constraints

1. **Daily Staffing Requirements**:
   - Monday: 1 employee
   - Tuesday: 2 employees
   - Wednesday: 1 employee
   - Thursday: 1 employee
   - Friday: 1 employee

2. **Employee Availability**:
   - Anna is **unavailable on Monday**.
   - Peter is **unavailable on Thursday and Friday**.

3. **Maximum Days per Employee**:
   - Mark can work **up to 4 days**.
   - All others (Anna, Kate, Peter) can work **up to 3 days**.

4. **Logical Constraint**:
   - **Mark can only work on a day if Anna is not working on that day.**
   - This models a mutual exclusion relationship between Mark and Anna.

#### Results

After solving the optimization model using PuLP, the program prints:
- The **status** of the solver (e.g., Optimal),
- Each employee's working days,
- The **total cost** of the schedule.

#### Purpose

This type of scheduling model is useful in workforce planning, particularly when:
- Employee availability varies,
- Cost optimization is crucial,
- Certain employees cannot work together (e.g., conflicts, safety rules).

It ensures staffing requirements are met at minimum cost, respecting operational and human resource constraints.



## Extension: Hourly Scheduling

We can extend the scheduling problem to assign specific hours rather than just days.

### Revised problem:
- Each employee can work up to 24 hours per week
- Hourly rates: Anna ($15), Kate ($16), Peter ($14)
- Tuesday requires 12 hours of work, other days require 8 hours
- Other constraints remain the same

This requires integer (not just binary) variables to represent hours worked.

In [ ]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpStatus, value, LpInteger

prob4 = LpProblem("Hourly_Scheduling_Problem", LpMinimize)

# Use integer variables for hours (0 to 24)
hours = LpVariable.dicts("Hours", (employees, days), lowBound=0, upBound=24, cat=LpInteger)

# Objective: minimize total cost
hourly_rates = [15, 16, 14]  # hourly rates
prob4 += lpSum([hourly_rates[i] * lpSum([hours[employee][day] for day in days])
               for i, employee in enumerate(employees)])

# Hours required each day
hours_required = [8, 12, 8, 8, 8]  # mon, tue, wed, thu, fri
for day, required in zip(days, hours_required):
    prob4 += lpSum([hours[employee][day] for employee in employees]) == required

# Availability constraints
prob4 += hours["Anna"]["mon"] == 0  # Anna can't work Monday
prob4 += hours["Peter"]["thu"] == 0  # Peter can't work Thursday
prob4 += hours["Peter"]["fri"] == 0  # Peter can't work Friday

# Maximum hours per week
for employee in employees:
    prob4 += lpSum([hours[employee][day] for day in days]) <= 24

# Solve and print the solution
print_solution(prob4)

In [ ]:
# Excercise 2

# 1) Extend the example to four Emploeeys: there is also Mark, who can work on any day and his rate is the lowest: $100 per day but he can work up to 4 days per week.

# 2) Add a constraint(s) that Mark can work 5h



In [ ]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpStatus, value, LpInteger

# Define employees and days
employees = ["Anna", "Kate", "Peter", "Mark"]
days = ["mon", "tue", "wed", "thu", "fri"]

# Define hourly rates for each employee
hourly_rates = [15, 16, 14, 10]  # Anna, Kate, Peter, Mark

# Define maximum weekly hours for each employee (adjust as needed)
max_hours_per_week = 20  # example limit for all employees

# Create problem
prob4 = LpProblem("Hourly_Scheduling_Problem", LpMinimize)

# Define decision variables: hours worked by each employee each day
hours = LpVariable.dicts("Hours", (employees, days), lowBound=0, upBound=24, cat=LpInteger)

# Objective: Minimize total cost
prob4 += lpSum([hourly_rates[i] * lpSum([hours[employee][day] for day in days])
               for i, employee in enumerate(employees)])

# Daily required hours
hours_required = [8, 12, 8, 8, 8]  # mon, tue, wed, thu, fri
for day, required in zip(days, hours_required):
    prob4 += lpSum([hours[employee][day] for employee in employees]) == required

# Availability constraints
prob4 += hours["Anna"]["mon"] == 0       # Anna can't work Monday
prob4 += hours["Peter"]["thu"] == 0      # Peter can't work Thursday
prob4 += hours["Peter"]["fri"] == 0      # Peter can't work Friday

# Weekly hour limit for all employees
for employee in employees:
    prob4 += lpSum([hours[employee][day] for day in days]) <= max_hours_per_week

# Mark's special constraints
# 1. He can only work 4 days max → binary variables to track which days he works
mark_day_work = LpVariable.dicts("MarkWorks", days, 0, 1, cat=LpInteger)

# If Mark works on a day, he must work exactly 5 hours
for day in days:
    prob4 += hours["Mark"][day] == 5 * mark_day_work[day]

# Binary variables: whether Anna works on a given day
anna_day_work = LpVariable.dicts("AnnaWorks", days, 0, 1, cat=LpInteger)

# Link binary variable to actual hours for Anna: if she works, hours > 0
for day in days:
    prob4 += hours["Anna"][day] <= 24 * anna_day_work[day]
    prob4 += hours["Anna"][day] >= 1 * anna_day_work[day]  # Optional: ensures if she works, it's at least 1h

# Mark's binary already defined and linked earlier, we just add:
# Constraint: Mark and Anna can't work on the same day
for day in days:
    prob4 += anna_day_work[day] + mark_day_work[day] <= 1


# Limit Mark to working on at most 4 days
prob4 += lpSum([mark_day_work[day] for day in days]) <= 4

# Solve the problem
prob4.solve()

# Print solution
print("Status:", LpStatus[prob4.status])
for employee in employees:
    print(f"\n{employee}'s schedule:")
    for day in days:
        print(f"  {day}: {hours[employee][day].varValue} hours")
print("\nTotal cost: $", value(prob4.objective))

Status: Optimal

Anna's schedule:
  mon: 0.0 hours
  tue: 0.0 hours
  wed: 0.0 hours
  thu: 0.0 hours
  fri: 0.0 hours

Kate's schedule:
  mon: 0.0 hours
  tue: 0.0 hours
  wed: 0.0 hours
  thu: 3.0 hours
  fri: 3.0 hours

Peter's schedule:
  mon: 8.0 hours
  tue: 7.0 hours
  wed: 3.0 hours
  thu: 0.0 hours
  fri: 0.0 hours

Mark's schedule:
  mon: 0.0 hours
  tue: 5.0 hours
  wed: 5.0 hours
  thu: 5.0 hours
  fri: 5.0 hours

Total cost: $ 548.0


### EXPLANATION

This problem expands upon a previous scheduling model by introducing **hour-based shifts** rather than binary workday assignments. The objective is to minimize the **total labor cost** while meeting daily workload demands and respecting each employee’s availability and contract constraints.

#### Problem Setup

We are scheduling 4 employees over 5 weekdays:
- **Employees**: Anna, Kate, Peter, Mark
- **Hourly wages**:
  - Anna: \$15/hour
  - Kate: \$16/hour
  - Peter: \$14/hour
  - Mark: \$10/hour

- **Daily hour requirements**:
  - Monday: 8 hours
  - Tuesday: 12 hours
  - Wednesday: 8 hours
  - Thursday: 8 hours
  - Friday: 8 hours

- **Weekly hour limit per employee**: 20 hours

#### Decision Variables

- `Hours[employee][day]`: Number of hours each employee works each day (integer between 0 and 24).
- `MarkWorks[day]`: Binary variable indicating whether Mark works on a given day.
- `AnnaWorks[day]`: Binary variable indicating whether Anna works on a given day.

#### Objective Function

The goal is to **minimize the total wage cost**, calculated as:

Total Cost = ∑ (Hourly Rate × Hours Worked)

#### Constraints

1. **Daily Hour Requirements**:
   - Ensure the total hours worked by all employees each day exactly meet the required number of hours.

2. **Employee Availability**:
   - Anna is not available on Monday.
   - Peter is not available on Thursday and Friday.

3. **Weekly Hour Limits**:
   - No employee can exceed 20 hours in the week.

4. **Special Constraints for Mark**:
   - Mark can only work a maximum of 4 days.
   - If Mark works on a day, he must work exactly **5 hours**.
   - Mark cannot work on the same day as Anna.
     - This constraint is modeled by ensuring `MarkWorks[day] + AnnaWorks[day] <= 1`.

5. **Binary Work Indicators for Anna**:
   - Used to enforce constraints regarding when Anna works and for linking her working status to actual hours.

#### Solution Output

After solving the problem using PuLP, the model prints:
- The solution **status** (e.g., Optimal),
- A detailed **schedule** for each employee (hours per day),
- The **total cost** of the schedule.

#### Applications

This model is useful in more realistic scheduling scenarios where:
- Employees work **partial shifts**,
- Labor cost control is crucial,
- Complex availability or exclusivity rules are present.

It demonstrates how to combine integer, binary, and logical constraints in a linear programming model.


## Conclusion

Integer programming is a powerful tool for solving optimization problems with discrete decisions. It's particularly useful in scheduling, where we often need to assign resources (like employees) to specific time slots subject to various constraints.

The key benefits include:
1. Ability to model logical conditions (AND, OR, IF-THEN)
2. Natural representation of indivisible resources
3. Optimal solutions for complex constraint satisfaction problems

However, integer programming problems can be computationally intensive as the number of variables increases.